# Web Analytics - Final Project
### Movie recommendations based on text from Wikipedia
_July 12, 2017_


## Group 1 Members:
* Mauricio Alarcon
* Sekhar Mekala
* Aadi Kalloo
* Srinivasa Illapani
* Param Singh

#### Background
Movies recommendation is one of the classic application of recommendation systems, and there are several ways to achieve this. In this project, our goal is to apply Natural Language Processing (NLP) techniques to the movie plot obtained from Wikipedia and determine relevant movies for a given movie. We scraped text related to 4037 movies from Wikipedia. These movies are American movies released since the year 2000. The key deliverables of this project are:

* Text corpus of 4037 movies
* Movie posters of 3749 movies
* Movie recommender based on movie's plot

#### Technologies used:
We used the following software/packages to develop the core logic of this project:
* Python 3
* Pandas
* Numpy
* Sklearn
* BeautifulSoup
* urllib

**NOTE:** We scraped movie release posters to render the recommendations in a more aesthetic fashion. However, we could not get all the movie posters, since some of them are not available, and some of them were not easily downloadable by our crawler since the webpage's HTML IDs are not consistent.

This project is divided into 4 logical phases:

1. Phase-1: Build a web crawler to download the movies text and release posters
2. Phase-2: Cleanse the text data to build the recommender system
3. Phase-3: Build the recommender system using the text data
4. Phase-4: Get the recommendations

The subsequent sections will have a detailed explanation of these phases.

# I. Phase-1: Build a Web Crawler

The main goal of **Phase-1** is to build a web crawler and scrape the text related to American movies, which were released between 2000 and 2016 years. Along with the text, we will also crawl the Movies posters. The major deliverables of this phase are:

1. Text (or plot) of American movies, which were released between the years 2000 and 2016
2. Movies release posters

NOTE: All the data will be obtained from Wikipedia.


## I.I Design

Wikipedia maintains list of movies, released in each year. The list of American movies released in each year are present at https://en.wikipedia.org/wiki/List_of_American_films_of_XXXX, where XXXX is the year. For each year between 2000 and 2016 (XXXX = 2000 to 2016), we have to recurrsively visit each year's URL to obtain the movies URLs, along with movie details such as cast, director, genre etc. Once the URLs related to all the movies are obtained, the web crawler will visit the movie URLs to scrape the plot of the movie. 

The following flowchart will provide an overview of the web crawling process:

<img src="crawler_logic.png">

 **Figure-1: Web crawling process**

The whole web crawling process is divided into 2 steps. In Step-1, we will visit Wikipedia to obtain a list of all the URLs related to the American movies which were released between the years 2000-2016. In Step-2, we will visit each of the URLs obtained in Step-1, to download movie text and release poster. There must be some delay of 2 to 3 seconds between successive requests to Wikipedia website.

The output of Step-1 will be a comma separated file (Movie_Details.csv), with the following details: 

**Movie** - Movie Name

**URL** - Wikipedia web page for the movie

**Year** - Year of release

**Director** - Director of the movie

**Cast** - Cast of the movie

**Genre** - Movie's genre

**Movie_ID** - Unique key to distinguish each movie

The output of Step-2 will be a set of text files, each file named using the Movie_ID, and a set of image files, each of the image files will also be named using the Movie_ID. Naming the files using the Movie_ID will help us to refer movie's text and image files uniquely.

# I.II Implementation

### I.II.I Import the required packages
Let us import all the required packages:

In [3]:
import pandas as pd
import numpy as np
from IPython.display import display # Allows the use of display() for DataFrames
import time
import pickle #To save the objects that were created using webscraping
import pprint
from lxml import html
import requests
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from urllib.request import urlopen
from bs4 import BeautifulSoup
from IPython.display import HTML
import re
import urllib
import os

### I.II.II Step-1 Implementation

In Step-1 of the web crawling process, we will obtain the list of movies, along with the URLs of the movies, for all the movies which were released between the years 2000 and 2016. 

But the challenge was, the format of the HTML file. Wikipedia used one format for the movies released between 2000 and 2013, and another format for the movies released in 2014-16. So we will sub-divide the Step-1 process further to scrape the list of movies between 2000 and 2013 in phase 1 and 2015-16 in phase2.

In [60]:
##PHASE-1: Get the movies and URLs for the years 2000-2013
#Define the lists to hold the details of the movies
URL = list()
Movie_Name = list()
Director = list()
Cast = list()
Genre = list()
year = list()

#Create a beautiful soup object
bs = BeautifulSoup(html)

#Iterate over the years 2000 to 2014.
for y in list(range(2000,2014)):
    
    #Prepare the URL String and open the URL
    url = "https://en.wikipedia.org/wiki/List_of_American_films_of_"+str(y)
    html = urlopen(url)
    
    #Mandatory wait of 3 seconds
    time.sleep(3)
    
    #Get the web page as HTML document
    bs = BeautifulSoup(html)
    
    #Parse and get the required data
    for table in bs.find_all('table', {"class":"wikitable"}):
        for row in table.find_all('tr'):
            columns = row.find_all('td')
            if len(columns) > 4:
                Movie_Name.append(columns[0].get_text())
                Director.append(columns[1].get_text())
                Cast.append(columns[2].get_text())
                Genre.append(columns[3].get_text())
                year.append(y)
                
                #Handle exceptions, so that the process continues
                try:
                    a = columns[0].find('a',href=True)['href']
                    URL.append("https://en.wikipedia.org"+a)
                except:
                    URL.append("NA")
                    continue


                

In [33]:
##PHASE-2: Get the movies details for the years 2014 to 2016
#Declare the lists
URL1 = list()
Movie_Name1 = list()
Director1 = list()
Cast1 = list()
Genre1 = list()
year1 = list()

#For the years between 2014 and 2016
for y in range(2014,2017):          
    
    #Prepare wiki URL
    url = "https://en.wikipedia.org/wiki/List_of_American_films_of_"+str(y)
    
    #Exception handling to ignore the failures and continue processing
    try: 
        html = urlopen(url)
    except:
        print("problem with the following URL...continuining...:")
        print(url)
        continue
    #Sleep for 3 secs    
    time.sleep(3)
    
    #Declare beautiful soup object
    bs = BeautifulSoup(html)
    for table in bs.find_all('table', {"class":"wikitable"}):
        for row in table.find_all('tr'):
            columns = row.find_all('td')
            if len(columns) > 3: #To make sure that we are accessing the movies tables only
                if len(columns) == 6:
                    #print(columns[0].get_text())
                    Movie_Name1.append(columns[0].get_text())
                    Director1.append(columns[1].get_text())
                    Cast1.append(columns[2].get_text())
                    Genre1.append(columns[3].get_text())
                    year1.append(y)
                    try:
                        a=columns[0].find('a',href=True)['href']
                        URL1.append("https://en.wikipedia.org"+a)
                    except:
                        URL1.append("NA")
                        continue
                
                if len(columns) == 7:
                    #print(columns[1].get_text())
                    Movie_Name1.append(columns[1].get_text())
                    Director1.append(columns[2].get_text())
                    Cast1.append(columns[3].get_text())
                    Genre1.append(columns[4].get_text())
                    year1.append(y)
                    
                    try:
                        a=columns[1].find('a',href=True)['href']
                        URL1.append("https://en.wikipedia.org"+a)
                    except:
                        URL1.append("NA")
                        continue

                if len(columns) > 7:
                    #print("col len:{}".format(len(columns)))
                    #print(columns[2].get_text())
                    Movie_Name1.append(columns[2].get_text())
                    Director1.append(columns[3].get_text())
                    Cast1.append(columns[4].get_text())
                    Genre1.append(columns[5].get_text())
                    year1.append(y)                    
                    try:
                        a=columns[2].find('a',href=True)['href']
                        URL1.append("https://en.wikipedia.org"+a)
                    except:
                        URL1.append("NA")
                        continue
                    

### I.II.III Save the results
We will save the movies details as a CSV file named Movie_Details.csv. This helps us to avoid running Step-1 again.

In [ ]:
#Create a data frame:
df = pd.DataFrame(list(zip(Movie_Name+Movie_Name1,URL+URL1,year+year1,Director+Director1,Cast+Cast1,Genre+Genre1)),\
                  columns=["Movie","URL","Year","Director","Cast","Genre"])

#Remove the rows which do not have URL information
df = df[df["URL"] != "NA"]

df["Movie_ID"] = df.index + 1

#Write the file
df.to_csv("Movies_Details.csv",encoding='utf-8',index=False)

### I.II.IV Read the saved file to a data frame.
We will read back the saved file in Step-1, to a data frame. You can download the Movies_Details.csv file from *https://goo.gl/RDhVtf*.

In [4]:
URL = pd.read_csv("Movie_Details.csv")

print("Initial rows of the file Movie_Details.csv")
display(URL.head())

print("The Movie_Details.csv has {} rows and {} columns".format(URL.shape[0],URL.shape[1]))

Initial rows of the file Movie_Details.csv


,Movie,URL,Year,Director,Cast,Genre,Movie_ID
0,102 Dalmatians,https://en.wikipedia.org/wiki/102_Dalmatians,2000,Kevin Lima,"Glenn Close, Gérard Depardieu, Alice Evans","Comedy, family",1
1,28 Days,https://en.wikipedia.org/wiki/28_Days_(film),2000,Betty Thomas,"Sandra Bullock, Viggo Mortensen",Drama,2
2,3 Strikes,https://en.wikipedia.org/wiki/3_Strikes_(film),2000,DJ Pooh,"Brian Hooks, N'Bushe Wright",Comedy,3
3,The 6th Day,https://en.wikipedia.org/wiki/The_6th_Day,2000,Roger Spottiswoode,"Arnold Schwarzenegger, Robert Duvall",Science fiction,4
4,Across the Line,https://en.wikipedia.org/wiki/Across_the_Line_...,2000,Martin Spottl,"Brad Johnson, Adrienne Barbeau, Brian Bloom",Thriller,5


The Movie_Details.csv has 4045 rows and 7 columns


There are 4045 movie URLs that have to be scraped from Wikipedia. Our goal is to scrape the image of the movie (if exists), along with the plot and initial introduction texts.

### I.II.V Step-2 Implementation

In Step-2 we will use the file Movie_Details.csv, which has the list of all movie URLs, along with some other details. Each URL of the movie will be crawled to extract the movies text and the images. In Step-2, we will build a set of functions to perform the web crawling. These functions are explained below:

#### I.II.V.I Functions
We will code the following functions to obtain the plot information of the movie, along with the release poster image of the movie.

* **Open_URL(url)** Gets the HTML content, prepares Beautiful Soup object and returns the Beautiful Soup object. The _url_ parameter represents the complete URL of the webpage to be scraped. If an error occurs while opeing the URL, then -1 is returned. If an error occurs while preparing the beautiful soup object, then -2 is returned.

* **Get_Plot(bs)** Takes a beautiful soup object as input. It extracts the introductory text, and the text in the section _plot_
If plot section is NOT present, then it return a negative code. The function returns the extracted text (in the _plot_ section and the initial paragraph). If an error occurs, then a negative code is returned.
    Return code = -1: If an error has occurred while getting the paragraphs from bs object
    Return code = -2: If there is NO _Plot_ section in the document
    Return code = -3: If an error occurred while extracting the first paragrapg in HTML doc

* **Get_All_Text(bs)** This function is called only when **Get_Plot(bs)** returns a -2 (No section with the heading _Plot_ is found). This function will take a beautiful object as input and gives all the text (present in < p > tags) as output. It will return -1, if any error occurs.

* **Save_Text_File(text,text_file_name)** It will save the text string (_text_) as a text file (with the name contained in *text_file_name*). The file is saved into the _data_ directory. Returns 0, if sucessfully saved. Returns -1, when the change directory command fails (while changing to _data_ sub-directory), and -2 when the change to parent directory from _data_ fails.

* **Get_And_Save_Image(bs,image_file_name)** It will get the movies poster (image file) and saves the image in the _images_ directory. It will take beautiful soup object as input and extracts the image URL. The image URL will be downloaded and saved to _images_ directory with the file name as the value present in *image_file_name*. Returns 0, if successfully downloaded and saved. Returns -1 if image is not found, and -2 if the an error occurs when saving the image.

* **Write_Error(url,msg,file)** Will write a error/warning message (contained in the parameter _msg_), while parsing the URL (present in the _url_ parameter). The _file_ parameter contains the name of the error file.  

The source code of these functions is given below:

In [9]:
def Open_URL(url):
    '''
    Gets the HTML content, prepares Beautiful Soup object and 
    returns the  Beautiful Soup object. 
    The url parameter represents the complete URL of the webpage.
    '''
    try:
         html = urlopen(url)
    except:
        return -1
    try:
        #bs = BeautifulSoup(html).encode("ascii")
        bs = BeautifulSoup(html)
        
        return bs
    except:
        return -2       

def Get_Plot(bs):  
    """
    Takes a beautiful soup object as input.
    Extracts the introductory text, and the text in the section plot
    If plot section is NOT present, just get all the available text in the webpage
    Returns the extracted text (if NO error, else returns a negative error code).
    -1: If an error has occurred while getting the paragraphs from bs object
    -2: If an error occurred while extracting the plot text
    -3: If an error occurred while extracting the first paragrapg in HTML doc
    """
    try:
        p = bs.find("p")
        initial_paragraph = p.getText()
    except:
        return -1

    # collect plot in this list
    plot = []
    
    # find the node with id of "Plot"
    try:
        mark = bs.find(id="Plot")
        # walk through the siblings of the parent (H2) node 
        # until we reach the next H2 node
        for elt in mark.parent.nextSiblingGenerator():
            if elt.name == "h2":
                break
            if hasattr(elt, "text"):
                plot.append(elt.text)
    except:
         return -2
    
    try:
        plot="".join(plot)
        text = initial_paragraph + plot
        return text
    except:
        return -3    
    

    
def Get_All_Text(bs):
    try:
        p = bs.find_all("p")
        l = list()
        for i in p:
            l.append(i.getText())
        return " ".join(l)
    except:
        return -1

def Save_Text_File(text,text_file_name):
    try:
        os.chdir("./data")
    except:
        return -1
    
    with open(text_file_name, 'w',encoding='utf-8') as f:
         f.write(text)
    
    try:
        os.chdir("..")
        return 0
    except:
        return -2

def Get_And_Save_Image(bs,image_file_name):    
    try: 
        img=bs.findAll("img",{"class":"thumbborder"})
        img_URL="https:"+img[0]['src']
    except:
        return -1
     
    try:
        os.chdir("./images")
        ignore=urllib.request.urlretrieve(img_URL,image_file_name)
        os.chdir("..")
        return 0
    except:
        return -2

def Write_Error(url,msg,file):
    with open(file,'a') as f:
        f.write("\n"+msg)
        f.write("\n"+url)

## I.III Beginning the major crawling process

The following code block will crawl the movies text from Wikipedia. This code ran for approximately 7 hours. So do NOT execute this code, unless you really want to start the download process. The output of this code is a series of text files and image files. The text files are saved to the _data_ directory and images to the _image_ directory. You can download these files directly from the location: *https://goo.gl/RDhVtf*

In [57]:
tracker = 0
term=1
start = time.time() # Get start time
print("Beginning the files download...")
#k = 1
for movie, url, year,Movie_ID in zip(list(URL["Movie"]),list(URL["URL"]),list(URL["Year"]),list(URL["Movie_ID"])):
    #if k < 30:
    #    k = k+1
    #    continue
    #print("{},{},{}".format(movie,url,year))
    #Open the URL
    
    bs=Open_URL(url)

    if bs == -1:
        Write_Error(url,"Error in opening the URL","error.txt")
        #print("Error in opening the URL: {}".format(url))
        continue
        

    if bs == -2:
        Write_Error(url,"Error in the creation of bs object for the URL","error.txt")
        #print("Error in the creation of bs object for the URL: {}".format(url))
        continue

    time.sleep(3)

    #create a name for the files
    #image_file_name = str(year)+"_"+movie.strip()+".jpg"
    #text_file_name =  str(year)+"_"+movie.strip()+".txt"
    image_file_name = str(Movie_ID)+".jpg"
    text_file_name =  str(Movie_ID)+".txt"

    text=Get_Plot(bs)
    
    if text == -1:
        Write_Error(url,"No paragraphs are found","error.txt")
        #print("No paragraphs are found")
        #print(url)
        continue

    if text == -2:
        Write_Error(url,"Warning: No Plot ID found","error.txt")
        #print("Warning: No Plot ID found")
        #print(url)
        
        text = Get_All_Text(bs)
        if text == -1:
            Write_Error(url,"No paragraphs are found","error.txt")
            #print("No paragraphs are found")
            #print(url)
            continue
        
    if text == -3:
        Write_Error(url,"Error while appending the main plot with the initial paragraph","error.txt")
        #print("Error while appending the main plot with the initial paragraph")
        #print(url)
        continue
    
    status = Save_Text_File(text,text_file_name)
    
    if status == -1:
        Write_Error(url,"Not able to change the directory to ./data","error.txt")
        #print("Not able to change the directory to ./data")
        #print(url)
        continue
    
    if status == -2:
        Write_Error(url,"Not able to change the directory to .. (parent directory) from ./data","error.txt")
        #print("Not able to change the directory to .. (parent directory) from ./data")
        #print(url)
        continue
        
    #Downloading Image files    
    status = Get_And_Save_Image(bs,image_file_name)
    
    if status == -1:
        Write_Error(url,"Not able to find the image","error.txt")
        #print("Not able to find the image")
        #print(url)
        continue

    if status == -2:
        Write_Error(url,"Not able to save the image","error.txt")
        #print("Not able to save the image")
        #print(url)
        continue

    #Check the status of the webbot    
    tracker = tracker + 1
    if (tracker % 100 == 0):
        print("Processed {} URLs".format(tracker))
        end = time.time() # Get end time
        elapsed_time = end - start
        print("Elapsed time to process 100 URLs:{} secs".format(elapsed_time))
        start = time.time() # Get end time
        #break

    #if term == 1:
    #    break

Beginning the files download...
Processed 100 URLs
Elapsed time to process 100 URLs:446.38204622268677 secs
Processed 200 URLs
Elapsed time to process 100 URLs:433.5029966831207 secs
Processed 300 URLs
Elapsed time to process 100 URLs:461.3205850124359 secs
Processed 400 URLs
Elapsed time to process 100 URLs:458.53809428215027 secs
Processed 500 URLs
Elapsed time to process 100 URLs:476.4255542755127 secs
Processed 600 URLs
Elapsed time to process 100 URLs:473.6112816333771 secs
Processed 700 URLs
Elapsed time to process 100 URLs:458.1378722190857 secs
Processed 800 URLs
Elapsed time to process 100 URLs:434.91878509521484 secs
Processed 900 URLs
Elapsed time to process 100 URLs:569.373973608017 secs
Processed 1000 URLs
Elapsed time to process 100 URLs:533.2878279685974 secs
Processed 1100 URLs
Elapsed time to process 100 URLs:493.94636368751526 secs
Processed 1200 URLs
Elapsed time to process 100 URLs:560.7728536128998 secs
Processed 1300 URLs
Elapsed time to process 100 URLs:519.74306

Out of 4045 movies we obtained 4037 movies text successfully. But we were able to obtain only 3749 images, since images were not available to some of the movies. The errors and warnings are logged into a file named _error.txt_. You can find this file at *https://goo.gl/RDhVtf*

## II. Phase-2: Data Cleansing
Now that we downloaded the data, let us clean the data to make the data ready for applying text analytics algorithms. To perform data cleansing, we created the following 3 functions:

* **Read_File(p)** It will open the input file, reads the text in the file, converts all the test to lower case, removes the punctuation (if any), and returns a list of tokens.

* **Remove_Stop_Words(tokens)** It will remove all stop words from the list of input tokens. Returns a refined list of tokens, with no stop words


* **Clean_Text(tokens)** It will clean all the text by removing any square brackets "[...]", braces "(" and ")", commas, colons, apostrophes etc. Returns a list of tokens that are just alphanumeric. 

The source code of these functions is given below:

In [3]:
def Read_File(p):
  with open(p, 'r',encoding='utf-8') as f:
    text = f.read()
    #Convert all the text to lower case
    #
    lowers = text.lower()
    #remove the punctuation using the character deletion step of translate
    no_punctuation = lowers.translate(string.punctuation)
    tokens = nltk.word_tokenize(no_punctuation)
    return tokens

def Remove_Stop_Words(tokens):
    filtered = [w for w in tokens if not w in stopwords.words('english')]
    return filtered



def Clean_Text(tokens):
    text = " ".join(tokens)
    #Remove punctuation marks, text in [], (, ), :
    filtered1 = re.sub('\.|\`|\'|\[.*\]|\(|\)|,|:', " ",text)
    
    #Remove any single characters
    filtered1 = re.sub('(^| ).( |$)', " ",filtered1)
    #Remove any contiguous spaces    
    filtered1 = re.sub(' +'," ",filtered1)
    
    #Include only alpha numeric characters
    filtered1=" ".join([i for i in filtered1.split() if re.search('[0-9 a-z]*',i)])
    return filtered1

The following code block will use the above functions to clean the text. Do NOT run this code, unless you want to test it, since it will run for some time. To save time, we saved the results as *processed_data.csv*. This file can be downloaded from *https://goo.gl/RDhVtf*.

In [6]:
import re
import string
import nltk
from collections import Counter
from nltk.corpus import stopwords

#List the files in the directory./data
file_names = os.listdir("./data")

#Process each file
file_names = [i for i in file_names if re.search('[1-9]*\.txt',i)]
y = list()
x = list()
k = 0
start = time.time()
for i in file_names:
    y.append(int(i.split(".")[0]))
    tokens = Read_File("./data/"+i)
    tokens = Remove_Stop_Words(tokens)
    cleaned_text = Clean_Text(tokens)
    x.append(cleaned_text)
    k = k+1
    if(k%100 == 0):
        temp_time = time.time() - start
        print("Processed {} files. Elapsed time:{} seconds".format(k, temp_time))
        
temp_time = time.time() - start
print("Processed {} files. Elapsed time:{} seconds".format(k, temp_time))


print("Now saving the result as processed_data.csv file...")
df=pd.DataFrame(list(zip(y,x)),columns = ["Movie_ID","Plot"])
df.to_csv("processed_data.csv",encoding='utf-8',index=False)        

Processed 100 files. Elapsed time:34.47499084472656 seconds
Processed 200 files. Elapsed time:68.88607907295227 seconds
Processed 300 files. Elapsed time:102.14953780174255 seconds
Processed 400 files. Elapsed time:134.21351504325867 seconds
Processed 500 files. Elapsed time:160.64091873168945 seconds
Processed 600 files. Elapsed time:190.43764638900757 seconds
Processed 700 files. Elapsed time:219.01563954353333 seconds
Processed 800 files. Elapsed time:251.53410053253174 seconds
Processed 900 files. Elapsed time:282.54603719711304 seconds
Processed 1000 files. Elapsed time:316.63187885284424 seconds
Processed 1100 files. Elapsed time:351.59121203422546 seconds
Processed 1200 files. Elapsed time:387.9956316947937 seconds
Processed 1300 files. Elapsed time:429.9995768070221 seconds
Processed 1400 files. Elapsed time:471.7074546813965 seconds
Processed 1500 files. Elapsed time:509.83835220336914 seconds
Processed 1600 files. Elapsed time:549.4117841720581 seconds
Processed 1700 files. E

We can see that the data cleaninsing process has ran for approximately 23 minutes. However the results of this process are saved as a CSV file processed_data.csv. This file is located at *https://goo.gl/RDhVtf*

Reading the processed_data.csv file into a data frame.

In [5]:
df = pd.read_csv("processed_data.csv")
print("Initial records of processed_data.csv file")
df.head()

Initial records of processed_data.csv file


,Movie_ID,Plot
0,1,102 dalmatians 2000 american family comedy fil...
1,10,american psycho 2000 american black comedy hor...
2,100,legacy 2000 american documentary film directed...
3,1000,lemony snicket series unfortunate events 2004 ...
4,1001,life death peter sellers 2004 british-american...


The above display shows that our final data frame, which will be used to build movies recommender is composed of two columns. The first column *Movie_ID* will uniquely identify the movie, and the *Plot* will identify the cleansed text of the movie plot. The movie's release poster will be present in the "./images" directory (you can download it from *https://goo.gl/RDhVtf*. But after download save it in the ./data directory or else this Jupyter notebook will not find the images)

## III. Phase-3: Building the recommender

Our recommender system is based on text analytics of the movies plot. We will use TF-IDF (Term Frequency - Inverse Document Frequency) score for each unique word in each document. All the unique words in the combined text of all the documents will form the *features*. 

Once the TFIDF is computed, we will obtain the cosine similarity between each pair of movies.

### III.I. TF-IDF Algorithm:

TFIDF (Term Frequency - Inverse Document Frequency) is one of the most popular text processing algorithms that helps us to accurately assign importance scores to each word in a document. 

At a very high level, the algorithm follows the below logic:

Let $D = {d_1, d_2 ... d_n}$ be a set of documents.

For each document $d$ in $D$ perform the following:

a. Get the frequencies of all the words in $d$. Call this as TF (Term Frequency) vector for document $d$

b. Get the list of all unique words in all the documents, and for each unique word, get the number of documents containing the word. Let DF (Document Frequency) be the vector containing these counts.

For each word $w$ in DF, get the following:

$$IDF_w=log(n/(1+\mbox{number of documents containing the word }w))$$ 

The log can have any valid base. 
IDF stands for Inverse Document Frequency. "n" represents the total number of documents

For each document $d$, multiply the elements of $TF_d$ with the corresponding elements of IDF, to obtain TFIDF vector for document $d$.


In sklearn package, we have TfidfVectorizer class, which implements the TF-IDF algorithm. Using this class, we are able to obtain the TF-IDF scores of all the unique words in all the movies plot.

### III.II Get the TFIDF scores

Using the data frame (*processed_data*), The below code block will get the TFIDF scores for all the words in each of the document.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df["Plot"])

print("The TF-IDF matrix has {} rows and {} columns".format(tfidf_matrix.shape[0],tfidf_matrix.shape[1]))

The TF-IDF matrix has 4037 rows and 54075 columns


The TF-IDF matrix for the movies text has 4037 rows (representing the number of movies) and 54075 columns (representing the unique words in all the movies text). Internally python represents this matrix as a sparse matrix, since most of the elements of this matrix have a value of 0.

### III.III. Get the cosine similarity measure between each pair of movie
To obtain the cosine similarity between each pair of movies, we will use cosine_similarity class of sklearn package. The below code block will compute the cosine similarity between each pair of movies.

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim_df = pd.DataFrame(cos_sim,columns=df["Movie_ID"].tolist(),index=df["Movie_ID"].tolist())

Let us display some rows and columns of the cosine similarity measure.

In [8]:
display(cos_sim_df)

,1,10,100,1000,1001,1002,1003,1004,1005,1006,...,990,991,992,993,994,995,996,997,998,999
1,1.000000,0.013110,0.025153,0.016308,0.011577,0.007299,0.010271,0.010509,0.006320,0.009129,...,0.011846,0.008216,0.011399,0.005237,0.007315,0.006427,0.008360,0.006734,0.002983,0.006313
10,0.013110,1.000000,0.009444,0.012178,0.004680,0.005145,0.009484,0.008657,0.011202,0.007491,...,0.013249,0.017382,0.007149,0.007385,0.011362,0.004854,0.019540,0.013429,0.000961,0.004964
100,0.025153,0.009444,1.000000,0.016322,0.059142,0.027863,0.014048,0.036151,0.007757,0.006688,...,0.042228,0.001339,0.009732,0.007993,0.013280,0.009319,0.021211,0.006464,0.023891,0.016957
1000,0.016308,0.012178,0.016322,1.000000,0.004812,0.024725,0.011021,0.018253,0.011957,0.012848,...,0.034581,0.014664,0.008955,0.009943,0.010786,0.008122,0.021269,0.020851,0.005010,0.009184
1001,0.011577,0.004680,0.059142,0.004812,1.000000,0.006460,0.007578,0.032768,0.004027,0.009007,...,0.009151,0.002705,0.014901,0.004565,0.016529,0.044348,0.009097,0.007988,0.014264,0.025655
1002,0.007299,0.005145,0.027863,0.024725,0.006460,1.000000,0.006180,0.015325,0.006960,0.030447,...,0.015545,0.013500,0.031731,0.008173,0.010223,0.007692,0.015265,0.008178,0.010762,0.013937
1003,0.010271,0.009484,0.014048,0.011021,0.007578,0.006180,1.000000,0.009379,0.007544,0.010232,...,0.027237,0.018090,0.007262,0.006958,0.008695,0.009016,0.010088,0.012814,0.011469,0.015597
1004,0.010509,0.008657,0.036151,0.018253,0.032768,0.015325,0.009379,1.000000,0.013861,0.013198,...,0.013786,0.027088,0.012796,0.010316,0.018577,0.009816,0.023062,0.009664,0.007165,0.012340
1005,0.006320,0.011202,0.007757,0.011957,0.004027,0.006960,0.007544,0.013861,1.000000,0.018479,...,0.011095,0.013903,0.015171,0.006398,0.006556,0.006096,0.010419,0.014563,0.004381,0.005944
1006,0.009129,0.007491,0.006688,0.012848,0.009007,0.030447,0.010232,0.013198,0.018479,1.000000,...,0.006813,0.011289,0.014773,0.010392,0.007992,0.007115,0.015426,0.007499,0.003727,0.018923


We can see that the cosine similarity matrix has 4037 rows and 4037 columns, and the elements represent the cosine similarity between each pair of movies. The diagonal elements of this matrix will be 1 since similarity score between the same move is always 1.

## IV. Phase-4: Getting recommendations

Let us build the required functions to make movie recommendations, given that the user has liked a movie.

### IV.I Functions
* **Get_Recommendations(Movie_ID,cos_sim_df)** This function will accept *Movie_ID*, and *cos_sim_df* as inputs. The *Movie_ID* is a number unique to a movie, and *cos_sim_df* is a pandas data frame containing the cosine similarity scores between all pairs of movies. This function will get top 6 Movies (based on the cosine similarity score between the input Movie_ID and other movies. Higher cosine similarity measure, better the match). The result is returned in the form of a dictionary.


* **Get_Available_Images()** This function will not accept any input. It returns the list of all movie IDs, for which we have an available image.


* **Display_Recommendations(Recommended_Movies_Dict,Movie_Map,Source_Movie_ID)** This function will accept 3 inputs. The *Recommended_Movies_Dict* is the dictionary of recommended movies (output of *Get_Recommendations(Movie_ID,cos_sim_df)* function). The *Movie_Map* is a data frame with the columns: *"Movie" (Movie name),"Movie_ID" (Unique ID),"URL" (Movie URL)*. This data frame is obtained by joining the *Movie_Details.csv* and *processed_data.csv* files data (using movie ID). This joining is needed, since it will help to map the movies which are successfully downloaded (4037 movies) and all the available movie names (4045 movies). The *Source_Movie_ID* is the movie ID, which is assumed to be liked by the user. The function does NOT return any value. It just renders the recommended movies along with the cosine similarity scores. The user can click on the movie to read visit the wikipedia site or hover on the image to get the text, which was used for building cosine similarity matrix.

The source code of these functions is given below:

In [9]:
#Get the mapping between available Movie plots and movie IDs
Movie_Map=pd.merge(URL[["Movie","Movie_ID","URL"]],df,how='inner',on=["Movie_ID"])[["Movie","Movie_ID","Plot","URL"]]


def Get_Recommendations(Movie_ID,cos_sim_df):
    #Get the indices (movie IDs) with highest cosine sim scores
    recommended_idx=np.argpartition(np.array(cos_sim_df[Movie_ID].tolist()), -6)[-6:]
    
    #Convert to a list
    Recommended_Movie_IDs = cos_sim_df.columns[recommended_idx].tolist()
    
    #Prepare a dict and return the recommended movies list
    return dict(zip(Recommended_Movie_IDs,np.array(cos_sim_df[Movie_ID].tolist())[recommended_idx]))


def Get_Available_Images():
    #Get all the available image names (movie IDs which have images)    
    image_files = os.listdir("./images")
    
    #Make sure that we are dealing with movie data files only
    image_files = [i for i in image_files if re.search('[1-9]*\.jpg',i)]
    
    #Define a list to collect the movie IDs
    y = list()
    for i in image_files:
        y.append(int(i.split(".")[0]))
    #Return the list    
    return y


def Display_Recommendations(Recommended_Movies_Dict,Movie_Map,Source_Movie_ID):
    #The following statement will make sure that we sort the movies in the descending order of similarity
    Recommended_Movies = pd.DataFrame(sorted(Recommended_Movies_Dict.items(), key=lambda x: -x[1]))[0].tolist()
    
    #Delete the liked movie from the list (since cosine sim with itself is 1)
    Recommended_Movies = Recommended_Movies[1:]
    
    Recommended_Movies_Plot = dict()
    Recommended_Movies_URL = dict()
    
    for i in Recommended_Movies:
        Recommended_Movies_Plot[i] = Movie_Map[Movie_Map["Movie_ID"] == i]["Plot"].tolist()[0]
        Recommended_Movies_URL[i] = Movie_Map[Movie_Map["Movie_ID"] == i]["URL"].tolist()[0]

    #Get the available movies with images    
    Available_Images_List = Get_Available_Images()
    
    Source_Movie_Name = Movie_Map[Movie_Map["Movie_ID"] == Source_Movie_ID]["Movie"].tolist()[0]
    Source_Plot = Movie_Map[Movie_Map["Movie_ID"] == Source_Movie_ID]["Plot"].tolist()[0]
    Source_URL = Movie_Map[Movie_Map["Movie_ID"] == Source_Movie_ID]["URL"].tolist()[0]
    print("Assuming that the user liked {}:".format(Source_Movie_Name))
    
    #Prepare HTML for display:    
    if Source_Movie_ID in Available_Images_List:
        display(HTML("<table><tr><td><a href='"+str(Source_URL)+\
                     "' target='_blank'><img src='./images/"+str(Source_Movie_ID)+".jpg' title='"+\
                     str(Source_Plot)+"'></a></td></tr></table>" \
            ))        
        
    display_html = ""
    display_values = ""
    for i in Recommended_Movies:
        if i in Available_Images_List:
            display_html = display_html + "<td><a href='"+str(Recommended_Movies_URL[i])+\
            "' target='_blank'><img src='./images/"+str(i)+".jpg' title='"+\
            str(Recommended_Movies_Plot[i])+"'></a></td>"
            display_values = display_values + "<td> Similarity:"+\
            str(Recommended_Movies_Dict[i])+"</td>"
    print("The following movies are recommended:")        
    display(HTML("<table><tr>"+display_html+"</tr><tr>"+display_values+"</tr></table>" \
            ))        

### IV.II Demonstration of the system
We will get recommended movies given that the user has liked some movies. The cosine similarity measure is also displayed, along with the movie recommendations. The recommended movies are sorted in the descending order of similarity score. Also the top 5 movies are displayed. At some places you may find less than 5 movies, since we avoided the display of the movie, if an associated image is not available (as the web robot did not download the picture due to unavailability or some other reason). Also if you hover over the image, you will see the text (cleansed) used for building the recommender, and if you click the image, you will redirected to the Wikipedia URL:

In [11]:
Recommended_Movies = Get_Recommendations(3974,cos_sim_df)
Recommended_Movies
Display_Recommendations(Recommended_Movies,Movie_Map,3974)

Assuming that the user liked X-Men: Apocalypse:


<img src='./images/3974.jpg' title='x-men apocalypse 2016 american superhero film based fictional x-men characters appear marvel comics ninth installment x-men film series sequel x-men days future past directed bryan singer screenplay simon kinberg story conceived singer kinberg michael dougherty dan harris film stars james mcavoy michael fassbender jennifer lawrence oscar isaac nicholas hoult rose byrne tye sheridan sophie turner olivia munn lucas till ancient mutant en sabah nur awakens 1983 plans wipe modern civilization take world leading x-men try stop defeat team renegade mutants en sabah nur powerful mutant believed first kind rules ancient egypt betrayed worshippers entomb alive four lieutenants die preserving awakening 1983 believes humanity lost way without presence aiming destroy remake world recruits cairo pickpocket ororo munroe control weather upgrades power in east berlin shape-shifting mutant raven investigates underground fight club discovers mutant champion angel possesses large feathered wings back kurt wagner teleport raven rescues kurt requests black marketeer caliban transport america en sabah nur recruits caliban enforcer psylocke leads angel en sabah nur enhances powers transforming angel wings metal alex summers discovers younger brother scott manifesting mutation shooting optic beams alex takes scott professor charles xavier educational institute westchester county new york hoping xavier hank mccoy teach control abilities scott meets telepathic telekinetic jean grey two develop attraction raven brings kurt institute en sabah nur powers cause disturbances around world leading xavier alex consult cia agent moira mactaggert researching legend nur in communist poland metal-controlling mutant erik lehnsherr lives wife young daughter nina en sabah nur resurrection causes worldwide earthquake uses powers save coworker prompting militia capture accidentally kill erik family retaliates executing en sabah nur later approaches devastated erik takes auschwitz erik power first manifested persuaded en sabah philosophy erik powers enhanced destroys camp joining en sabah nur en sabah nur remotely accesses cerebro brainwave-amplifying device xavier uses locate mutants forces telepathic xavier make global superpowers launch entire nuclear arsenals space prevent interference en sabah nur plan new four horsemen arrive mansion kidnap xavier attempting stop alex accidentally causes explosion destroys mansion peter maximoff realized erik biological son hoping xavier help find arrives time uses super-speed evacuate students teachers explosion destroys building alex apparently killed process colonel william stryker forces subsequently capture hank raven peter moira take military facility interrogation scott jean kurt follow covertly liberate comrades using stryker mind-controlled brainwashed experiment weapon whose memories jean partially restores at en sabah nur behest erik uses powers control earth magnetic poles causing destruction across planet sabah nur plans transfer consciousness xavier body use xavier power enslave every person earth xavier secretly sends telepathic distress call jean others travel cairo battle sabah nur horsemen rescue xavier loses hair process nears completion angel defeated erik ororo persuaded turn en sabah nur help scott keep occupied physically xavier fights telepathically astral plane finally xavier encourages jean unleash full extent powers incinerating en sabah nur killing good psylocke escapes xavier moira rekindle relationship erik jean help reconstruct school erik refuses xavier offer stay help teach peter decides tell erik yet erik son using confiscated sentinels hank raven train new x-men recruits scott jean ororo kurt peter in post-credits scene men black suits visit weapon facility retrieve data stryker mutant research including x-ray blood sample marked weapon behalf essex corporation'>


The following movies are recommended:


In [33]:
Recommended_Movies = Get_Recommendations(1,cos_sim_df)
Display_Recommendations(Recommended_Movies,Movie_Map,1)

Assuming that the user liked  102 Dalmatians:


<img src='./images/1.jpg' title='102 dalmatians 2000 american family comedy film directed kevin lima live-action directorial debut produced edward feldman walt disney pictures sequel 1996 film 101 dalmatians stars glenn close reprising role cruella de vil attempts steal puppies grandest fur coat yet close tim mcinnerny two actors first film return sequel however film nominated academy award best costume design lost gladiator three years prison cruella de vil cured desire fur coats dr pavlov released custody probation office provision forced pay remainder fortune eight million pounds dog shelters borough westminster repeat crime cruella therefore mends working relationship valet alonzo lock away fur coats cruella probation officer chloe simon nevertheless suspects partly chloe owner now-adult dipstick one original 15 puppies previous film dipstick mate dottie recently given birth three puppies domino little dipper oddball lacks spots mend reputation cruella buys second chance dog shelter owned kevin shepherd resolve financial insolvency verge eviction meanwhile dr pavlov discovers therapy subjects subjected loud noises revert original states conceals discovery big ben rings presence cruella reverts former personality enlists help french furrier jean-pierre lepelt steal 102 dalmatian puppies new fur coat kevin tells chloe cruella violates parole entire fortune go since dog shelter one borough westminster cruella kevin framed theft puppies invites chloe dinner lepelt steals dottie three puppies dipstick hurries back apartment hides lepelt truck later captured train station chloe rushes home save pets arrives late joined kevin escaped prison help dogs talking parrot waddlesworth upon finding ticket orient express paris dropped lepelt kevin chloe attempt fail stop cruella lepelt oddball waddlesworth pursue enemies secretly paris kevin chloe save captive puppies seen locked cellar puppies flee cruella goes puppies alone alonzo scolded beyond patience enough abused defeats lepelt frees kevin chloe give chase wedding cake factory puppies kevin dogs imprison cruella immense cake lepelt thereupon arrested kevin chloe personally awarded remnants cruella fortune alonzo oddball coat develops spots'>


The following movies are recommended:


<img src='./images/659.jpg' title='101 dalmatians ii patch london adventure 2003 american direct-to-video animated adventure musical comedy-drama film written directed jim kammerud brian smith released walt disney home entertainment february 25 2003 based dodie smith novel hundred one dalmatians sequel 1961 disney animated film 101 dalmatians features voices martin short jason alexander barry bostwick susanne blakeslee kath soucie jeff bennett jim cummings bobby lockwood garnered dvdx awards best animated feature best director best editing best musical score disney re-released september 16 2008 radcliffe family one hundred one dalmatians preparing move dalmatian plantation home countryside plenty room however one puppies patch feels ignored wishes unique like television hero thunderbolt watching thunderbolt adventure hour hears chance appear show accidentally left behind family leaves plantation decides head audition meet hero win guest spot show fails impress producers allowing jealous lil lightning manipulate making hero show depressed patch leaves runs thunderbolt elsewhere london cruella de vil returned probation restraining order past behavior longer allowed maintain previous lifestyle attempts soothe fixation spots help lars german artist order inspire restarts hunt dalmatians using newspaper picture patch find new address patch family finally becomes aware missing go back london find cruella bails former henchmen jasper horace prison sends stolen dog food truck steal remaining puppies succeed dealing nanny take lars cruella requests made masterpiece fur lars refuses wanting harmed angered bound gagged returns original plan making dalmatian fur coat the captured puppies use twilight bark send distress signal picked patch thunderbolt set save patch family way encounter lil lightning follows convinces thunderbolt use patch stealth plan openly attack frightens jasper horace cruella knocks unconscious patch captured lil lightning believing scheme fulfilled sneaks building reveals thunderbolt fraud tells said ago lie get way finally shadow fleeing crushed revelation thunderbolt telling patch really actor patch falls despair siblings restore spirit reminding thunderbolt escaped similar predicament one tv episodes patch breaks releases family thunderbolt remains behind ashamed disappointing patch dalmatians escape building roof cruella panther de ville villains stalled meanwhile thunderbolt escapes cage frees lars the puppies steal double-decker bus cruella jasper horace discover escape pursue stolen truck chase ensues streets london crashing filming lil lightning new show process driving window cruella jasper horace finally corner dogs alley patch tries hold others escape undaunted luckily thunderbolt arrives driven scene lars fakes heart attack briefly gaining lil lightning sympathy causing cruella distraction knock jasper horace incapacitate patch retreats back bus puts reverse sending cruella jasper horace lil lightning scrambling river thames along stolen truck thunderbolt stand triumphantly patch finally brings bark similar thunderbolt the police arrest lil lightning jasper horace cruella driven completely insane sent mental institute radcliffes nanny told jasper horace cruella made arrive along patch parents introduces thunderbolt thank dismisses simply actor patch hand eyes real one kind wonder-dog newspaper montage reveals fates characters post-credits scene shows thunderbolt tv show patch new sidekick puppies extras chasing villain away sunset'>,<img src='./images/3564.jpg' title='cold comes night 2013 american crime thriller film directed tze chun co-wrote script oz perkins nick simon released september 20 2013 uk january 10 2014 united states film stars alice eve bryan cranston logan marshall-green film produced mynette louie trevor sagan chloe single mother living daughter sophia operates motel topo blind man traveling cross country jeep associate john stop chloe motel john hires prostitute gwen convinces topo stay

In [34]:
Recommended_Movies = Get_Recommendations(3934,cos_sim_df)
Display_Recommendations(Recommended_Movies,Movie_Map,3934)

Assuming that the user liked London Has Fallen:


The following movies are recommended:


In [35]:
Recommended_Movies = Get_Recommendations(2635,cos_sim_df)
Display_Recommendations(Recommended_Movies,Movie_Map,2635)

Assuming that the user liked Paranormal Activity 2:


<img src='./images/2635.jpg' title='paranormal activity 2010 american found footage supernatural horror film directed tod williams written michael perry film prequel/sequel 2007 film paranormal activity beginning two months following events depicted original film released theaters midnight october 22 2010 united states united kingdom canada mexico brazil argentina poland ireland august 2006 burglary occurs home kristi sprague grayden daniel rey brian boland entire house trashed except infant son hunter bedroom daniel installs security cameras throughout house vandalism capture many strange occurrences martine vivis cortez hispanic family housekeeper nanny goes hunter room hearing loud bang senses demonic presence room takes downstairs attempts cleanse house evil spirits daniel arrives home catches burning sage martine fired kristi believes home haunted tells daniel daniel reviews footage dismisses claims kristi talks sister katie katie featherston tormented demon little girls daniel daughter ali molly ephraim begins investigating mysterious occurrences discovers humans make deals demons wealth power forfeiting soul first-born son also discovers hunter first male born kristi side since 1930s the violence quickly escalates family german shepherd abby becomes aware demon presence attacked daniel ali take abby veterinarian leaving kristi alone hunter kristi checks baby demon assaults drags basement stays hour finally basement door opens possessed kristi walks following day ali home kristi get bed finds basement door covered scratches word meus latin mine etched it ali goes upstairs check hunter sees kristi strange bite mark leg ali tries get hunter kristi furiously orders touch terrified ali begs daniel come home arrives shows footage kristi attack immediately calls martine prepares cross exorcise demon tells kristi memory possessed since curse transferred blood relative daniel tells ali going pass demon onto katie kristi hunter safe ali begs unfair katie daniel sees way save wife son that night daniel tries use cross kristi attacks houselights go using handheld camera night vision finds kristi hunter disappeared furniture starts topple chandeliers begin shake daniel chases kristi basement attacks touches cross collapses daniel hears demonic growls finally shaking stops daniel puts kristi bed burns photo young katie photo micah sloat later finds attic katie house first film three weeks later katie visits explains strange things started happening house returns home boyfriend micah october night micah killed possessed katie breaks daniel kristi home kills daniel snapping neck watches tv kills kristi hunter room violently hurling camera takes baby leaves cradling hunter screen fades black hunter crying turns laughter an epilogue text states ali school trip found bodies daniel kristi upon return home katie hunter whereabouts remain unknown'>


The following movies are recommended:


In [36]:
Recommended_Movies = Get_Recommendations(2810,cos_sim_df)
Display_Recommendations(Recommended_Movies,Movie_Map,2810)

Assuming that the user liked Kung Fu Panda 2:


<img src='./images/2810.jpg' title='kung fu panda 2011 3d american computer-animated comedy-drama martial arts film directed jennifer yuh nelson produced dreamworks animation distributed paramount pictures sequel 2008 film kung fu panda second installment kung fu panda franchise years events first film lord shen gary oldman scion peacock clan rules gongmen city ancient china seeks weaponize gunpowder recently invented create fireworks discovering court goat soothsayer michelle yeoh warrior black-and-white defeat change ways shen leads army wolves exterminate panda population avert prophecy shen parents horrified atrocity exile son swears revenge about twenty years later po jack black living dream dragon warrior protecting valley peace alongside friends fellow kung fu masters furious five teacher shifu dustin hoffman tells however yet achieve inner peace defending village wolf bandits stealing refined metal shen po distracted symbol wolf leader danny mcbride armor causes po flashback mother allows wolves escape po asks goose father mr ping james hong origins ping reveals found po infant radish crate adopted po remains unsatisfied wondering ended valley peace shifu receives word shen returned gongmen city killed thundering rhino leader kung fu council protecting city plotting conquer china newly developed weapon cannon shifu sends po furious five gongmen city stop shen destroy weapon find city occupied shen forces two surviving council members storming ox croc imprisoned six heroes ask council members help liberate city two cite helplessness shen weapon refuse po five discovered wolf boss give chase arrested front shen tower upon brought shen tower po five free destroy shen weapon however po distracted flashback upon seeing symbol shen plumage realizing shen last night po saw parents allows shen escape destroy tower arsenal cannons po five escape tigress angelina jolie confronts po distraction reluctantly explains remembers shen presence night separated parents wants answers past shen though empathetic tigress tells po stay behind safety despite po breaks factory question shen inadvertently foiling five plan destroy factory shen claims po parents abandoned blasts po factory river cannon presumed dead po survives rescued soothsayer takes ruins nearby village po born guided soothsayer embrace past po remembers infant parents sacrificed save shen army mother hiding radish crate luring shen forces away po attains inner peace realizing lived happy fulfilling life despite early tragedy po returns gongmen city save captive five prevent shen conquest china battle ensues shen armada sailing heart gongmen city po five joined shifu persuaded ox croc help po modifies movements used inner peace training redirect shen cannonballs armada destroying po urges shen let go past shen attacks po shen slashes ropes holding last cannon falls top crushing death victorious po returns valley peace reunites mr ping lovingly declaring goose father at time po birth father li fred tatasciore revealed survived shen attack shown living far-off hidden village inhabited surviving pandas senses son still alive'>


The following movies are recommended:


In [37]:
Recommended_Movies = Get_Recommendations(2656,cos_sim_df)
Display_Recommendations(Recommended_Movies,Movie_Map,2656)

Assuming that the user liked Saw VII:


<img src='./images/2656.jpg' title='saw 3d also known saw final chapter simply saw vii 2010 american 3d horror film directed kevin greutert written patrick melton marcus dunstan starring tobin bell costas mandylor betsy russell sean patrick flanery cary elwes seventh originally intended final installment saw franchise film series 3d two men brad ryan wake metropolitan storefront window bound wrists opposite sides worktable circular saw front mutual lover dina suspended third saw scene draws attention police arrive puppet informs men kill let dina die initially fight brad cut ultimately choose allow dina die instead jigsaw widow jill tuck meets matt gibson internal affairs detective working precinct detective mark hoffman works incriminates hoffman exchange protection immunity prosecution gibson distrusted hoffman ever since saved life gunning homeless man agrees terms meanwhile hoffman abducts skinhead gang dan evan jake kara places junkyard trap kills leaving reverse bear trap scene incriminate jill later put protective custody meeting jigsaw survivors including lawrence gordon cauterized leg steam pipe escaping bathroom uses cane prosthetic foot hoffman abducts bobby dagen self-help guru achieved fame fortune falsifying survival story sends gibson videos cryptic clues game location offering end games jill given him dagen awakens cage abandoned asylum told wife joyce die t save within one hour joyce chained neck steel platform gradually pulls watches dagen progress escapes cage dangles floor spikes begins searching joyce along way finds nina publicist fish hook trap suzanne lawyer piercing wheel cale mcnamara best friend co-conspirator hanging trap represent three wise monkeys killed despite dagen efforts save removing upper wisdom teeth obtain combination security door dagen finds joyce forced reenact trap claimed survive must drive hooks pectoral muscles hoist ceiling deactivate trap fails nonetheless forced watch brazen bull capsule closes around joyce incinerates death gibson eventually discovers game location sends swat team sealed another room killed toxic gas simultaneously two officers infiltrate hoffman command center junkyard find dan corpse sitting front several monitors connected police headquarter security cameras sees hoffman brought morgue body bag tries warn palmer automatic turret gun rises kills along men hoffman kills dr heffner palmer everyone else path jill briefly escapes caught knocked unconscious straps chair locks reverse bear trap head awakens timer starts killed trap activates tearing face apart bleeding profusely after destroying workshop hoffman attacked subdued three pig-masked figures one gordon flashbacks show john kramer found unconscious steam pipe nursed back health gordon assisted ever since known jill delivered package hospital videotape kramer asked watch jill take action anything happened end gordon hoffman brought disused bathroom shackled ankle pipe gordon appears throws hacksaw used sever foot door turns lights seals door leaving hoffman die'>


The following movies are recommended:


<img src='./images/2448.jpg' title='saw 2009 american horror film directed kevin greutert screenplay written patrick melton marcus dunstan sixth installment seven–part saw franchise stars tobin bell costas mandylor betsy russell mark rolston peter outerbridge shawnee smith produced mark burg oren koules twisted pictures distributed lionsgate two predatory lenders awaken either side room split caged-in scale wearing head harnesses screws poised pierce temples one sets flesh weight upon tray survive eddie cuts fat torso simone chops arm eddie killed timer expires reviews game footage detective lieutenant mark hoffman called scene fbi agent dan erickson found peter strahm fingerprints also present lindsey perez whose survival covered erickson protection learn dr adam heffner coroner performed autopsies case thus far knife used cut jigsaw piece eddie flesh used seth baxter agents intend find baxter videotape prove strahm involvement while hospital meet simone hoffman met reporter pamela jenkins claims found something meatpacking plant later meets jill tuck clinic takes control games given five envelopes box left john kramer containing photos people tested leaves flashback reveals kramer brought amanda young jill deemed lost cause proof methods later flashback jill delivers package box hospital shows jill met kramer plant hours death gave box key promise would protected games ended william easton insurance executive umbrella health meets debbie company attorney harold abbott succumbed heart disease coverage request denied due application discrepancy hoffman abducts easton office night brings abandoned zoo janitor hank awaken vises designed tighten around torsos time breathe oxygen masks strapped faces kramer appears videotape informs easton one hour complete four tests remove bombs strapped limbs lose family hank heavy smoker unable hold breath long killed vise easton tests force save workers traps including hanging room steam room carousel shown kramer met easton opening jill clinic took issue formula cancer diagnosed kramer coverage request experimental norwegian treatment rejected easton the game viewed woman named tara son brent cage observation room contains tank hydrofluoric acid switch marked live die pamela awakens another cage side room game hoffman called away erickson perez found baxter videotape unscrambled offsite technician inform abnormalities found strahm fingerprints erickson eventually reveals knowledge strahm death moment voice unscrambled hoffman attacks kills everyone room plants fingerprints using strahm severed hand lights room fire destroy evidence he returns observation room finds letter written amanda used knowledge role jill miscarriage blackmail killing lynn denlon letter found pamela given jill uses ambush hoffman easton reaches end path one second left finding cages revealed pamela sister tara brent harold family tara informed videotape kramer either kill easton free using switch unable make decision brent angrily shifts switch bed needles swings injects easton body acid killing jill restrains hoffman locks new reverse bear trap head shows sixth envelope contains photo leaves 45 seconds key hoffman escapes breaking hand jamming trap activates pulls head free ripping open cheek'>,<img src='./images/2213.jpg' title='saw 2008 canadian-american horror film directed david hackl written patrick melton marcus dunstan stars tobin bell costas mandylor scott patterson betsy russell mark rolston julie benz carlo rota meagan good fifth installment saw franchise released october 23 2008 australia october 24 2008 north america convicted murderer seth baxter wakes chained table beneath pendulum blade videotape informs release crushing hands two presses blade still swings cuts half someone watches hole wall in meatpacking plant fbi agent peter strahm guns jeff denlon locked sickroom discovers hidden exit microcassette recorder urges stay sickroom ignores attacked tunnel pig-masked figure wakes head sealed box quickly 

In [38]:
Recommended_Movies = Get_Recommendations(3176,cos_sim_df)
Display_Recommendations(Recommended_Movies,Movie_Map,3176)

Assuming that the user liked Titanic 3D:


<img src='./images/3176.jpg' title='titanic 1997 american epic romance-disaster film directed written co-produced co-edited james cameron fictionalized account sinking rms titanic stars leonardo dicaprio kate winslet members different social classes fall love aboard ship ill-fated maiden voyage in 1996 treasure hunter brock lovett team aboard research vessel akademik mstislav keldysh search wreck rms titanic necklace rare diamond heart ocean recover safe containing drawing young woman wearing necklace dated april 14 1912 day ship struck iceberg rose saved with rose hiding cal en route rms carpathia takes survivors new york city rose gives name rose dawson later finds cal committed suicide losing money 1929 wall street crash back present lovett decides abandon search hearing rose story alone stern keldysh rose takes heart ocean possession along drops sea wreck site seemingly asleep died bed photos dresser depict life freedom adventure inspired life wanted live jack young rose reunites jack titanic grand staircase applauded died'>


The following movies are recommended:


<img src='./images/1125.jpg' title='ballad jack rose 2005 drama film written directed rebecca miller starring husband daniel day-lewis also stars camilla belle catherine keener paul dano ryan mcdonald jason lee jena malone susanna thompson beau bridges film tells story environmentalist teenage daughter live secluded island commune filmed rock barra prince edward island canada new milford connecticut jack slavin daniel day-lewis scottish farmer heart ailment lives island hippie commune decades struggling keep landowners building developments wetland teenaged daughter rose camilla belle beautiful isolated girl passion gardening since rose mother left family jack homeschooled daughter expose life beyond small island home jack believes need woman around travels mainland ask girlfriend kathleen catherine keener move jack breaks news shocked rose kept relationship secret rose remains disdainful kathleen two teenage sons move in kathleen struggles adapt slavin rural lifestyle sons rodney ryan mcdonald thaddius paul dano almost polar opposites thaddius sullen rude delinquent rodney insecure often overlooked still strained relationship kathleen rose develops strange bonds new step-brothers clear thaddius attracted rose like one night rose spies jack kathleen bed together develops strange jealousy toward kathleen rose decides lose virginity shocks rodney confronting topless asking sex rodney refuses reasons instead ends giving dramatic haircut afterward rose calmly takes father shotgun possibly misfires jack kathleen bedroom sleep initially shocked jack confronts rose disbelief two seem forget event within minutes kathleen asks jack relationship rose might psychological problems dealt jack denies daughter problems meanwhile rose rodney become good friends rodney often criticized mother overweight two fight constantly diet rose sees kindness intelligence however still mission lose virginity rose thoughts turn thaddius trapping copperhead intended scaring kathleen rose sees thaddius girl named red berry sex woods later night thaddius enters rose room though dislikes rose allows sex copperhead rose kept cage bed escapes house lock cage loosened drops open vibrations bed thaddius rose sex in to irk father rose hangs bloodied bedsheet front yard jack furious daughter ruined gives thaddius one day move meanwhile kathleen cornered copperhead resulting chaos puts whole household edge night rose holds screening homemade movie hippie commune treehouse film rolls thaddius advances rose stopped jack scuffle thaddius falls treehouse rushed hospital rose runs away hides days jack finally finds offers ask kathleen leave would bring rose home distraught kathleen demands 20 000 exchange leaving jack pays jack returns rose hideout overjoyed news kathleen gone night rose kisses jack shock daughter love makes upset weeps waking next morning memory kiss haunts jack rose go house builder marty rance jack breaks finding fight left tells rance destroy wetlands rose return home jack dies within days rose originally planned kill father died setting house fire lying next jack body changes mind escapes two years later rose shown living vermont working greenhouse gray father friend island works well'>,<img src='./images/2566.jpg' title='greatest 2009 american drama film written directed shana feste directorial debut starring pierce brosnan also executive producer susan sarandon carey mulligan michael shannon eighteen-year-old bennett brewer aaron johnson rose mulligan sex together time either ever done bennett killed truck crashes car parked middle road night family feel like go mother grace sarandon father allen brosnan get unexpected visitor knocking door turns rose pregnant bennett child story develops bennett younger brother ryan introduced grieving fact say final good bye brother one sees true relationships family story comes together grace waits bed jordan walker man driving truck collided bennett car crash waiting wake ask occurred 17 minutes crash death fin

In [39]:
Recommended_Movies = Get_Recommendations(3893,cos_sim_df)
Display_Recommendations(Recommended_Movies,Movie_Map,3893)

Assuming that the user liked The Martian:


<img src='./images/3893.jpg' title='martian 2015 english-language science fiction film directed ridley scott based andy weir 2011 novel name matt damon stars astronaut mistakenly presumed dead left behind mars film depicts struggle survive others efforts rescue film ensemble cast also features jessica chastain kristen wiig jeff daniels michael peña kate mara sean bean sebastian stan donald glover aksel hennie chiwetel ejiofor in 2035 crew ares iii mission mars exploring acidalia planitia martian solar day sol 18 31-sol expedition strong dust storm threatens topple mars ascent vehicle mav forcing abort mission evacuation astronaut mark watney struck debris lost storm last telemetry suit indicates signs life watney believed dead storm worsening second mission commander melissa lewis orders crew take return orbiting vessel hermes watney awakens storm low oxygen warning makes way hab short habitat crew base operations removes piece antenna destroyed suit biomonitor lodged torso performing self-surgery begins video diary realizes chance rescue rendezvous ares iv crew schiaparelli crater 3 200 kilometres 2 000 mi away four years immediate concern enough food survive long watney botanist improvises farm inside hab utilizing martian soil fertilized human waste water produced extracting hydrogen leftover rocket fuel potatoes intended thanksgiving dinner also begins modify functional rover longer journeys preparation long distance travel back earth nasa holds funeral reviewing satellite photos mars satellite planner mindy park sees evidence watney activities realizes survived nasa administrator teddy sanders public relations director annie montrose mission director vincent kapoor release news despite objections hermes flight director mitch henderson sanders decides inform ares iii crew believing would distract them watney takes rover retrieve pathfinder probe fell silent 1997 using lander camera establishes rudimentary communication earth first plastic bin lids using hexadecimals nasa instructs watney modify rover code partially link pathfinder communicate via text watney becomes angry learns crew told survival openly expresses sanders reluctantly lets henderson finally inform them henderson jpl director bruce ng prepare space probe deliver enough food last watney ares iv arrival hab airlock malfunctioned due small puncture destroys watney crops sanders orders team speed supply mission iris probe skipping safety inspections however oversight causes destabilization subsequent explosion rocket 40 seconds liftoff causing watney think seemingly inevitable death the china national space administration offers nasa taiyang shen classified booster rocket carry payload mars meanwhile jpl astrodynamicist rich purnell devises trajectory divert hermes back mars rescue two years earlier using chinese booster resupply hermes additional eighteen months sanders rejects plan refusing risk crew henderson surreptitiously sends details hermes crew unanimously vote plan nasa powerless stop resupplies hermes uses earth gravity slingshot back mars watney begins 90-sol journey schiaparelli pre-positioned ares iv mav rendezvous hermes watney removes many components mav including nose cone lighten replacing piece hab canvas angered vincent kapoor much mav however reach needed speed altitude canvas rips lewis forced use maneuvering thrusters change course explosive decompression hermes internal atmosphere reduce speed even enough commander lewis uses manned maneuvering unit still reach watney watney pierces glove pressure suit uses escaping air propel towards lewis effectively rescuing alone 560 sols mars after returning earth watney becomes survival instructor astronaut candidates five years later occasion ares mission launch involved watney rescue begun new lives'>


The following movies are recommended:


In [40]:
Recommended_Movies = Get_Recommendations(4015,cos_sim_df)
Display_Recommendations(Recommended_Movies,Movie_Map,4015)

Assuming that the user liked Sully:


<img src='./images/4015.jpg' title='sully also known sully miracle hudson 2016 american biographical drama film directed clint eastwood written todd komarnicki based autobiography highest duty chesley sullenberger jeffrey zaslow film stars tom hanks sullenberger aaron eckhart laura linney anna gunn autumn reeser holt mccallany jamey sheridan jerry ferrara supporting roles film follows sullenberger january 2009 emergency landing us airways flight 1549 hudson river 155 passengers crew survived minor injuries subsequent publicity investigation on january 15 2009 us airways pilots captain chesley sully sullenberger first officer jeff skiles board us airways flight 1549 laguardia airport charlotte douglas international airport three minutes flight approximate altitude 800 feet approx 850 airbus a320 strikes flock birds disabling engines without engine power judging unable reach nearby airports teterboro airport closest sully ditches aircraft hudson river crew passengers evacuate without casualty press public hail sullenburger hero incident leaves symptoms ptsd dream plane crashes building sully learns preliminary data acars suggest port engine still running idle power theoretically would left enough power return laguardia land teterboro national transportation safety board claims several confidential computerized simulations show plane could landed safely either airport without engines sully however insists lost engines left without sufficient time speed altitude land safely airport sully realizes board believes accident may pilot error would end career arranges simulations rerun live pilots results relayed public hearing simulations result successful landings one airport sully argues unrealistic pilots knew advance situation would face suggested emergency action able practice scenario several times board accepts real life pilots would taken time react run emergency checks deciding divert plane the two simulations rerun relayed hearing time allowing 35-second pause plane diverted simulated diversion laguardia ends plane landing short runway teterboro crash buildings airport board announces analysis port engine recovered river confirms sully account disabled bird strikes board concludes sullenberger acted correctly selecting best options available event saved lives everyone aboard'>


The following movies are recommended:


<img src='./images/1698.jpg' title='snakes plane 2006 american action thriller film directed david ellis starring samuel jackson released new line cinema august 18 2006 north america film written david dalessandro john heffernan sheldon turner follows events hundreds snakes released passenger plane attempt kill trial witness after witnessing gangster eddie kim brutally beat s prosecutor daniel hayes death hawaii sean jones escorted fbi agents neville flynn john sanders boeing 747-400 testify trial los angeles despite increased security flight kim arranges time-release crate full venomous snakes placed cargo hold attempt bring plane reaches los angeles international airport lax ensure snakes attack passengers without need provocation one henchmen disguised airport ground employee spray passengers leis special pheromone makes snakes highly aggressive crate opens midway flight snakes make way cabin couple sex bathroom man using bathroom first killed plane captain sam mckeon investigates fixes electrical short killed viper caused co-pilot rick unaware snake believes sam suffered heart attack continues toward lax some snakes attack rick fending accidentally releases oxygen masks throughout plane causing several snakes drop cabin numerous passengers including agent sanders killed snakes invade cabin surviving passengers made way front plane put blockades luggage desperate attempt stop snakes rick attacked plane starts dip downwards causing food trolley crash luggage blockade passengers flee upstairs first class cabin blocking stairwell inflatable liferaft agent flynn flight attendant claire regain control plane rick retakes controls flynn go cargo hold restore air conditioning/ventilation system agent flynn contacts fbi special agent hank harris ground gets touch ophiologist dr steven price customs main source animal smuggling cases based pictures reptiles emailed via passenger mobile phone price believes los angeles snake dealer known illegally importing exotic highly dangerous snakes responsible shootout dealer bitten one snakes harris withholds antivenom give details dealer finally reveals kim hired obtain snakes adds latter managed smuggle board plane price injects antivenom injured dealer commandeers dealer supply antivenom plane victims based list given harris gives orders eddie kim arrested multiple counts murder attempted murder harris contacts flynn telling anti-venom ready passengers land however flynn discovers cockpit filled snakes rick dead brief discussion troy three gs bodyguard agrees land plane based prior flight experience everyone gets prepared flynn shoots two windows pistol causing plane depressurize snakes blown cockpit lower floor plane flynn troy take controls plane troy reveals flight experience video game flight simulator despite lack real-world experience troy makes emergency landing plane makes terminal passengers leave plane anti-venom given need flynn sean disembark plane final snake jumps bites sean chest flynn draws gun shoots snake paramedics rush sean unharmed due bulletproof vest token gratitude sean later takes flynn bali teaches surf'>,<img src='./images/1737.jpg' title='united 93 2006 biographical drama-thriller film written co-produced directed paul greengrass chronicles events aboard united airlines flight 93 majority film presented real time on morning september 11 2001 passengers board united airlines flight 93 bound san francisco newark liberty international airport including tom burnett mark bingham todd beamer jeremy glick richard guadagno louis nacke ii william joseph cashman patrick joseph driscoll four al-qaeda terrorists ziad jarrah saeed al-ghamdi ahmed al-nami ahmed al-haznawi also board flight meanwhile newly promoted faa national operations manager ben sliney meeting reported american airlines flight 11 boston los angeles hijacked mohamed atta overheard radio saying planes minutes later aircraft crashes north tower world trade center much sliney staff horror learn second flight un

In [41]:
Recommended_Movies = Get_Recommendations(3077,cos_sim_df)
Display_Recommendations(Recommended_Movies,Movie_Map,3077)

Assuming that the user liked The Hunger Games:


The following movies are recommended:


In [42]:
Recommended_Movies = Get_Recommendations(616,cos_sim_df)
Display_Recommendations(Recommended_Movies,Movie_Map,616)

Assuming that the user liked Spider-Man:


<img src='./images/616.jpg' title='spider-man 2002 american superhero film directed sam raimi based marvel comics character name film stars tobey maguire peter parker high school student living new york city turns crimefighting developing spider-like super powers spider-man also stars kirsten dunst peter love interest mary jane watson willem dafoe norman osborn/green goblin rosemary harris cliff robertson aunt may uncle ben james franco best friend harry osborn high-school senior peter parker school outcast bully victim school field trip visits genetics laboratory friend harry osborn unwitting love interest mary jane watson peter bitten genetically engineered super spider shortly arriving home becomes ill falls unconscious meanwhile harry father scientist norman osborn owner oscorp attempting secure important military weapons contract experiments unstable performance-enhancing chemical absorbing chemical goes insane kills assistant the next morning peter discovers longer near-sighted body metamorphosized muscular physique school finds body produce webs wrists quickened reflexes enable avoid injury confrontation flash thompson brushing uncle ben advice great power comes great responsibility peter considers impressing mary jane car enters underground wrestling tournament wins first match promoter cheats prize money thief suddenly raids promoter office peter allows escape revenge moments later discovers ben carjacked shot dying peter arms overcome anger vengeance peter corners subdues tries kill carjacker hesitates sees thief let escape thief attempts flee trips falls window death instead meanwhile oscorp chief competitor quest aerospace weapon test norman kills several scientists army general slocum upon graduating peter begins using abilities fight crime donning costume dubbed press spider-man j jonah jameson newspaper chief editor hires peter freelance photographer since person providing clear images spider-man writes stories implicate spider-man crimes actually stopping norman upon learning oscorp board members plan sell company attacks world unity fair though peter intervenes norman still kills board members jameson quickly dubs norman green goblin norman tells norman despite good done people eventually come despise offers peter place side gives time think meanwhile meeting peter mary jane followed four men alley corner attempt mug peter comes rescue thanks kissing afterwards peter runs norman refuses offer fight peter wounded whilst mocking son affection mary jane thanksgiving dinner norman sees wound peter realizes secret identity shortly afterwards mary jane admits peter infatuated spider-man rescued twice asks peter whether spider-man ever asked harry loves mary jane arrives interprets feelings peter defeated harry laments father peter loves mary jane unintentionally revealing spider-man weakness norman attacks may later kidnaps holds mary jane roosevelt island tram car full children hostage alongside queensboro bridge challenging norman another confrontation forces peter choose save drops peter manages save mary jane tram car norman pelted heckled civilians sided peter goblin redirects fight abandoned building engage lengthy brutal battle norman manages overpower norman latter reveals halt fight begs forgiveness time programs glider impale foe behind sensing danger peter instinctively dodges glider impales norman dies norman asks peter tell harry green goblin identity harry arrives find spider-man standing father body incorrectly believes murdered father at norman funeral harry swears vengeance toward spider-man asserts peter family left mary jane confesses peter loves peter however feels must protect unwanted attentions peter enemies hides true feelings informs mary jane friends'>


The following movies are recommended:


<img src='./images/3635.jpg' title='amazing spider-man also released amazing spider-man rise electro markets film stars andrew garfield peter parker spider-man emma stone gwen stacy dane dehaan green goblin harry osborn campbell scott embeth davidtz peter parents sally field aunt may addition new cast including paul giamatti rhino aleksei sytsevich jamie foxx electro max dillon in 2000 oscorp scientist richard parker records video message explain disappearance later wife mary aboard private jet hijacked assassin sent kill richard wife richard mary fight man ensuing struggle causes plane crash killing richard mary uploads video in present day richard son peter continues fight crime spider-man pursues apprehends aleksei sytsevich later peter meets gwen high school graduation ceremony insisting keep vow father ends relationship peter childhood friend harry osborn returns manhattan see terminally-ill father norman ceo oscorp norman explains illness genetic harry age first develops norman gives harry small device claims contains life work next day norman dies harry appointed new oscorp ceo while working oscorp laboratory electrical engineer max dillon shocks accident falls tank genetically-engineered electric eels attack mutates living electric generator meanwhile peter attempts maintain friendship gwen reveals may move england schooling discuss dillon wanders times square accidentally causing blackout stopped peter battle dillon taken ravencroft institute studied german scientist dr kafka meanwhile first symptoms harry illness showing uses device norman gave deduce spider-man blood could help save asks peter selling photos spider-man daily bugle aid finding spider-man peter refuses unsure effects transfusion would although later comes harry spider-man still refuses harry becomes resentful towards spider-man oscorp board-members particular vice-president donald menken frame harry covering dillon accident remove ceo overthrow take control oscorp harry assistant felicia hardy informs equipment could help makes deal dillon calls electro get back inside oscorp building finds suit armour equipment made norman well venom now-destroyed genetically-altered spiders venom accelerates harry illness transforms goblin-like creature suit built-in emergency protocol restores health cures disease peter uses information left father locate video message abandoned subway station hidden lab richard explains leave refused cooperate norman biogenetic weaponization plans peter hears voicemail gwen telling offered british scholarship heading airport earlier expected manages catch professes love vowing go wherever goes agree go england together electro causes another blackout peter heads fight spider-man gwen follows together restore power overload electro body killing him the transformed harry calls green goblin arrives equipped norman armor weaponry upon seeing gwen deduces spider-man secret identity swearing revenge refused blood transfusion kidnaps takes top clock tower peter manages defeat harry webbing breaks battle gwen falls death peter depressed distraught mourns gwen death ends career spider-man five months later harry coping aftereffects transformation incarcerated ravencroft associate gustav fiers visits harry pair discuss forming team harry orders fiers start sytsevich later unknown team men break sytsevich prison equipped electromechanical suit armor sytsevich calls rhino rampages streets peter inspired gwen graduation speech resumes role spider-man confronts'>,<img src='./images/2017.jpg' title='spider-man 2007 american superhero film based fictional marvel comics character spider-man directed sam raimi screenplay raimi ivan raimi alvin sargent final film raimi original spider-man film trilogy following spider-man 2002 spider-man 2004 film stars tobey maguire kirsten dunst james franco thomas haden church topher grace bryce dallas howard rosemary harris j simmons james cromwell cliff robertson final acting appearance death 2011 following eve

In [11]:
Recommended_Movies = Get_Recommendations(2708,cos_sim_df)
Recommended_Movies
Display_Recommendations(Recommended_Movies,Movie_Map,2708)

Assuming that the user liked The Adventures of Tintin: The Secret of the Unicorn:


<img src='./images/2708.jpg' title='adventures tintin known adventures tintin secret unicorn outside north america cast includes jamie bell andy serkis daniel craig nick frost simon pegg see also crab golden claws synopsis secret unicorn synopsis red rackham treasure synopsisin 1955 17-year-old journalist tintin dog snowy browsing outdoor market brussels belgium tintin buys miniature model ship unicorn accosted interpol officer barnaby dreaded ship collector ivan ivanovitch sakharine unsuccessfully try buy model tintin tintin takes ship home accidentally broken incident snowy cat apartment parchment scroll slips rolls furniture tintin noticing meanwhile bumbling police detectives thomson thompson trail pickpocket aristides silk after visit maritime library search clues tintin returns find unicorn stolen visits sakharine marlinspike hall accuses theft sees miniature model unicorn notices sakharine model broken realizing two unicorn models returning home snowy shows scroll reading old message written interrupted arrival barnaby assassinated attempting recover unicorn one last time tintin places scroll wallet stolen silk next morning scuffle police detectives later tintin abducted accomplices sakharine imprisoned ss karaboudjan learns sakharine formed alliance ship staff led mutiny take control board tintin meets captain haddock ship captain haddock permanently drunk thus unaware happenings board ship tintin haddock snowy eventually outrun crew escape karaboudjan lifeboat ship fails ram boat instead rammed empty lifeboat captain accidentally released escape presuming survived number lifeboats sakharine sends seaplane find capture trio seize plane use fly towards fictitious moroccan port bagghar however seaplane crashes desert due low fuel while trekking desert haddock hallucinates remembers facts ancestor sir francis haddock 17th-century captain unicorn sir francis treasure-laden ship attacked crew pirate ship led red rackham eventual surrender sir francis sank unicorn treasure prevent falling rackham hands implies three unicorn models containing scroll together scrolls reveal coordinates location sunken unicorn treasure the third model bagghar possessed omar ben salaad sakharine causes distraction concert results successfully stealing third scroll chase city gains scrolls men toss captain haddock snowy water force tintin go instead saving scrolls boat leaves tintin ready give persuaded haddock continue help officers thompson thomson tintin haddock track sakharine revealed descendant red rackham head back port set trap sakharine uses pistol resist arrest men fail save sakharine challenges haddock final showdown sakharine haddock sword-duel cranes swords eventually resulting sakharine defeated pushed overboard haddock climbing ashore sakharine arrested thomson thompson guided three scrolls indicating location marlinspike hall tintin haddock find treasure clue unicorn location men agree setting expedition find shipwreck'>


The following movies are recommended:


<img src='./images/2169.jpg' title='meet dave 2008 american comedy science fiction film directed brian robbins written bill corbett rob greenberg stars eddie murphy elizabeth banks gabrielle union ed helms scott caan kevin hart new york city apartment young boy named josh morrison austyn lind myers stares telescope object falling sky golf-ball-sized metal ball flies window lands fishbowl quickly draining water along goldfish decides show school science class presentation some months later massive fireball crashes water near liberty island revealed spaceship resembles human eddie murphy controlled 100 tiny humanoid aliens captain also played murphy pilots spaceship command deck located head help second-in-command number ed helms researcher number gabrielle union spaceship looks human aliens t know act society ship displays numerous superpowers superstitious cop named dooley scott caan desperately searches alien the aliens need save planet nil energy crisis need salt plan take draining earth oceans using metal ball recover spaceship hit josh single mother gina morrison elizabeth banks car captain decides befriend gina josh telling name dave ming cheng based quick scan common earth names gina comment looks like dave ming cheng see missing ball photograph taken science presentation breakfast gina dave goes josh school pretends substitute teacher eventually able talk josh alone josh tells ball taken bully nicholas berman josh help dave takes metal ball back bully the captain via dave spends time josh gina realizes humans advanced thought feelings love witnessing gina painting homeless man offering share blanket dave sleeps doorway decides cancel plan drain oceans would destroy life earth police track dave using impression face found dirt crash site arrest spending much time earth crew begins exhibit new feelings adopting earth culture mannerisms general laid-back attitude number decides captain rest crew changing behavior unacceptable takes command imprisons captain number command dave breaks police station try arrest number become infatuated captain becomes jealous gina first cooperates command change later agrees captain view humans caught number expelled spaceship meantime number 17 kevin hart young fun-loving alien jumps drunk alcohol dave imbibed captain apologizes number ignoring admits loves wants back police station dooley discovers number 17 coffee interrogates find dave going number takes dave harbor tries throw metal orb ocean stopped captain number managed gain reentry back onto ship convince rest crew real captain charge reinstated orders number stuck ship butt forever metal orb meanwhile slips dave hand rolls ocean captain attempts retrieve orb told enough power either retrieve return home captain decides save earth rest crew agrees ball thrown ocean number retrieved dave powers dooley partner catch point guns power dave shields disabled leaving crew defenseless josh tries tell police officers dave harmless ignored grabs dooley taser uses dave recharging captain number reveal police officers stand captain says goodbye josh gina saying understands love number 17 returned dave dooley fly away team fbi arrives throws net dave fbi agents wrestle body dave crew evacuates one ship lifeboat shoes activate engines detaches shoe heads home nil leaving behind ship number lifeboat captain asks number hand marriage accepts kiss'>,<img src='./images/640.jpg' title='treasure planet 2002 american animated science fiction film produced walt disney feature animation released walt disney pictures november 27 2002 43rd disney animated feature film film science fiction adaptation robert louis stevenson adventure novel treasure island first film released simultaneously regular imax theaters film employs novel technique hand-drawn 2d traditional animation set atop 3d computer animation on planet montressor young jim hawkins enchanted stories legendary pirate captain flint ability appear nowhere raid passing ships disappear order hide loot 

In [15]:
Recommended_Movies = Get_Recommendations(2354,cos_sim_df)
Recommended_Movies
Display_Recommendations(Recommended_Movies,Movie_Map,2354)

Assuming that the user liked The Hangover:


<img src='./images/2354.jpg' title='hangover 2009 american comedy film directed todd phillips co-produced daniel goldberg written jon lucas scott moore film stars bradley cooper ed helms zach galifianakis heather graham ken jeong rachael harris mike epps justin bartha jeffrey tambor tells story phil wenneck stu price alan garner doug billings travel las vegas bachelor party celebrate doug impending marriage however phil stu alan wake doug missing memory previous night events must find groom wedding take place to celebrate upcoming marriage tracy garner doug billings travels las vegas best friends phil wenneck stuart stu price tracy brother alan doug future father-in-law vintage mercedes-benz w111 spend night caesars palace relax hotel suite celebrating drinks hotel rooftop next morning phil stu alan awaken find memory previous night doug nowhere found stu tooth alan pocket suite state complete disarray tiger bathroom chicken living room baby closet name carlos find doug mattress impaled statue outside caesars palace ask mercedes valet delivers lvpd police cruiser using clues retrace steps trio travel hospital discover drugged rohypnol roofies causing memory loss came hospital chapel chapel learn stu married stripper named jade despite long-term relationship abusive domineering girlfriend melissa outside chapel trio attacked gangsters saying looking someone flee visit jade discovering mother baby whose real name tyler arrested police stealing police cruiser told mercedes impounded trio released unknowingly volunteer targets taser demonstration driving mercedes discover naked chinese man named leslie chow trunk chow jumps trunk beats trio crowbar flees alan confesses drugged drinks ensure good night believing drug ecstasy returning suite find boxer mike tyson orders trio return tiger mansion immediately stu drugs tiger remaining rohypnol drive towards tyson home mercedes-benz however tiger awakens midway attacks clawing phil neck damaging car interior pushing car rest way mansion tyson shows trio footage house help locate doug driving car intentionally t-boned another vehicle passengers revealed gangsters chapel boss chow chow accuses trio kidnapping stealing 80 000 purse trio denies chow tells doug threatens kill money returned unable find chow $ 80 000 alan help stu jade uses knowledge card counting win 82 400 playing blackjack they meet chow exchange money find doug black drug dealer accidentally sold alan rohypnol wedding set occur hours phil calls tracy tells find doug black doug mentions take roofies re likely end floor roof stu realizes doug trio travels back hotel find badly sunburned doug roof stu phil alan moved mattress night practical joke forgot left doug thrown mattress onto statue attempt signal help leaving stu makes arrangements go date jade following week less four hours wedding flights a available four drive home along way doug reveals possession chow original 80 000 despite late arrival doug tracy married former learns damaged mercedes wedding gift stu breaks melissa grown tired controlling attitude reception ends alan finds stu digital camera containing photos events remember four agree look pictures together deleting evidence exploits save alan traumatized see'>


The following movies are recommended:


<img src='./images/2780.jpg' title='hangover part ii 2011 american comedy film produced legendary pictures distributed warner bros pictures sequel 2009 film hangover second installment hangover trilogy directed todd phillips co-wrote script craig mazin scot armstrong film stars bradley cooper ed helms zach galifianakis ken jeong jeffrey tambor justin bartha paul giamatti tells story phil stu alan doug travel thailand stu wedding bachelor party las vegas stu takes chances opts safe subdued pre-wedding brunch things go planned resulting another bad hangover memories previous night two years events first movie las vegas stu price phil wenneck alan garner doug billings travel thailand celebrate stu upcoming wedding lauren much alan dismay joined lauren younger brother teddy rehearsal dinner lauren father reveals disapproval stu toast later night stu hesitantly joins phil doug alan teddy beer sitting campfire roasting marshmallows group toast stu lauren future happiness the next morning phil stu alan along gangster leslie chow alan befriended las vegas chain-smoking capuchin monkey awaken dirty hotel room bangkok stu face tattoo replica mike tyson alan head completely shaved find teddy discover severed finger chow begins relay events prior night seemingly dies snorting line cocaine panicked trio dispose chow body ice box through tip doug still resort left campfire earlier others go police station pick teddy given wheelchair-bound elderly buddhist monk knows happened however refuses reveal anything taken vow silence finding business card travel smoldering ruins business enter nearby tattoo parlor stu got tattoo learn started fight escalated riot trio returns monk buddhist temple encouraged head monk meditate alan able recall white lion strip club learn stu sodomized kathoey prostitute named kimmy upon exiting trio attacked two russian mobsters steal monkey one shoots phil arm after phil treated clinic alan confesses drugged marshmallows previous night muscle relaxers adhd medication order sedate teddy accidentally mixed bags phil stu become furious alan drugged stu blames alan ruining life attacks phil breaks tells stick together notice something alan stomach address time meeting meet gangster named kingsley demands chow bank account password next morning exchange teddy return hotel try find chow password discover still alive steal monkey code given put inside jacket safe-keeping chow back russian mobsters violent car chase monkey shot injured taking code leaving monkey veterinary clinic group completes deal kingsley suddenly interpol agents appear arrest chow kingsley turns undercover agent tells trio police used information teddy disappeared arrest chow kingsley actually know teddy is desperate clues phil calls doug wife tracy tell ca t find teddy stu decides call wedding live bangkok say teddy died alan playing game pac-man power goes stu suddenly realizes teddy trio rushes back hotel find teddy elevator unharmed albeit still missing finger teddy woken middle night get ice severed finger first bucket ice melted became trapped power went four use chow speedboat perfect life keys teddy pocket travel back wedding reception arriving land lauren father cancel wedding stu makes defiant speech rejects boring instead states fact quite wild impressed lauren father gives couple blessing exchange vows alan presents stu special gift post-reception dance musical guest performance mike tyson teddy later discovers taken many pictures night cell phone group along tyson agree look pictures together reveal teddy lost finger playing knife game erasing evidence exploits'>,<img src='./images/3338.jpg' title='hangover part iii 2013 american comedy film produced legendary pictures distributed warner bros pictures third final installment hangover trilogy film stars bradley cooper ed helms zach galifianakis justin bartha ken jeong supporting cast includes jeffrey tambor heather graham mike epps melissa mccarthy john goodman todd phillips directing screen

In [17]:
Recommended_Movies = Get_Recommendations(3166,cos_sim_df)
Recommended_Movies
Display_Recommendations(Recommended_Movies,Movie_Map,3166)

Assuming that the user liked Taken 2:


<img src='./images/3166.jpg' title='taken 2012 english-language french action thriller film directed olivier megaton stars liam neeson maggie grace famke janssen rade šerbedžija leland orser jon gries d sweeney luke grimes 2011 funeral son marko associates tropojë albanian mafia head freelance terrorist murad hoxha vows seek vengeance son killer traveling paris men interrogates tortures former intelligence agent jean-claude pitrel whose business card found scene marko death finds information bribes corrupt police official pitrel files deduces pitrel old friend bryan mills responsible istanbul meanwhile bryan finished three-day security job wealthy arabian sheikh istanbul surprised ex-wife lenore daughter kim going lunch lenore next day bryan spots murad men following tells lenore run tries shake albanians finally surrenders capture lenore realizing kim also target bryan calls hotel tells hide captured knocked narrowly avoids capture kidnappers forced flee shooting two security guards regaining consciousness bryan finds zip tied pipe head empty basement uses concealed miniature cell phone hidden sock contact kim instruct alert american embassy instead convinces let help opening father equipment case kim takes grenade detonates nearby rooftop resulting sound allows bryan instruct triangulating location the mobsters bring lenore cut throat slightly hang upside bleed soon leave bryan frees next kim detonate two grenades releases steam chimney guide location kim tosses gun chimney bryan uses kill guards rescues kim watches lenore get recaptured stealing taxi bryan kim pursue kidnappers van whereupon suv driven one kidnappers henchmen arrives distract chase shootout ensue alerting turkish police ending bryan manages lure suv path oncoming train taking out leaving kim s embassy bryan uses memory find murad hideout rescues lenore pursues surviving mobsters bathhouse kills confronting murad bryan offers let walk agrees return home cease desire revenge murad agrees bryan drops gun former goes back word tries kill bryan find gun unloaded realizing murad never drop vendetta bryan kills impaling onto sharp towel hook three weeks later mills family diner back los angeles milkshakes celebrate kim passing driving test joined much bryan surprise kim boyfriend jamie jokingly tells father shoot one'>


The following movies are recommended:


<img src='./images/2239.jpg' title='taken 2008 english-language french action thriller film directed pierre morel written luc besson robert mark kamen starring liam neeson maggie grace leland orser jon gries david warshofsky holly valance katie cassidy xander berkeley olivier rabourdin gérard watkins famke janssen retired cia field operative bryan mills liam neeson attempts build closer relationship shy daughter kim maggie grace lives mother lenore famke janssen wealthy stepfather stuart xander berkeley overseeing helping provide security concert pop star sheerah holly valance bryan saves armed attacker gratitude sheerah offers kim assessed singer bryan tell kim asks father permission travel paris best friend amanda katie cassidy initially refuses eventually agrees lenore pressures airport learns girls actually planning follow band u2 european tour upon arriving charles de gaulle international airport kim amanda meet young man named peter nicolas giraud offers share taxi learn kim amanda go amanda cousins apartment kim find cousins spain kim makes call father witnesses amanda abducted living room kim follows father instructions hide bedroom listen closely dragged underneath bed kim yells description abductor bryan suspects one abductors picked phone states release daughter go kidnappers warns failure result hunt wherein death outcome man albanian accent answers good luck sam leland orser old friend former colleague bryan deduces kidnappers part albanian sex trafficking ring identifies man marko hoxha reveals based previous history kim disappear good found within 96 hours using stuart private plane bryan flies paris investigates apartment finds peter reflection picture kim phone finds peter airport tries capture fleeing peter hit killed truck with lead dead bryan turns old contact french former intelligence agent jean-claude pitrel olivier rabourdin desk job agency jean-claude informs local red-light district albanian prostitution ring operates warns get involved however aid translator bryan trails infiltrates makeshift brothel construction yard finds young woman wearing kim denim jacket firefight mobsters takes girl hotel clears drugs system the following morning bryan returns heated conversation jean-claude girl awakens tells safe house albanians keep abductees posing pitrel enters house pretense business renegotiation police protection costs bryan identifies marko man spoke phone wherein violent fight ensues resulting deaths gangsters building minus marko quick search reveals several heavily drugged girls including dead amanda bryan tortures marko information using makeshift electric chair marko explains high value virgins black market kim sold quickly marko gives buyer patrice saint-clair gérard watkins bryan leaves die agony continuous electrocution later evening bryan visits pitrels dinner discovering jean-claude corruption coerces providing saint-clair location wounding wife arm gunshot bryan infiltrates secret sex slave auction beneath saint-clair manor soon kim comes sale forces abil arab bidder purchase making way knocked chained pipe manages escape eliminate saint-clair henchmen bryan shoots saint-clair reveal kim whereabouts wherein latter reveals yacht owned sheikh named raman killed gunshots bryan boards raman yacht dispatches guards well abil revealed raman underboss enters raman suite find raman holding kim knife-point raman attempts negotiate bryan shoots kim reunited mother stepfather film ends bryan kim visiting sheerah'>,<img src='./images/3777.jpg' title='taken sometimes stylized tak3n 2014 english-language french action thriller film directed olivier megaton written luc besson robert mark kamen third final installment taken trilogy film stars liam neeson forest whitaker maggie grace famke janssen film set three years second film in 2014 former covert operative bryan mills liam neeson visits daughter kim maggie grace deliver birthday gift awkward visit invites ex-wife lenore famke janssen dinner although de

In [18]:
Recommended_Movies = Get_Recommendations(3604,cos_sim_df)
Recommended_Movies
Display_Recommendations(Recommended_Movies,Movie_Map,3604)

Assuming that the user liked 300: Rise of an Empire:


<img src='./images/3604.jpg' title='300 rise empire 2014 american epic historical fantasy war film directed noam murro follow-up 2007 film 300 taking place main events film based battle artemisium battle salamis based as-yet-unreleased frank miller graphic novel xerxes zack snyder directed co-wrote original film acts writer producer rise empire queen gorgo sparta tells men battle marathon king darius persia killed general themistocles athens ten years earlier darius son xerxes witnesses father death advised continue war since gods defeat greeks darius naval commander artemisia claims darius last words fact challenge sends xerxes journey desert xerxes finally reaches cave bathes otherworldly liquid emerging 8-feet tall god-king returns persia declares war greece avenge father as xerxes forces advance towards thermopylae themistocles meets council convinces provide fleet engage persians sea themistocles travels sparta ask king leonidas help informed dilios leonidas consulting oracle gorgo reluctant side athens themistocles later reunites old friend scyllas infiltrated persian troops learned artemisia born greek defected persia family raped murdered greek hoplites taken sex slave subsequently left dead streets rescued adopted persian emissary lust vengeance gained attention king darius made naval commander killed many enemies themistocles also learns leonidas marched fight persians 300 men themistocles leads fleet fifty warships several thousand men include scyllas scyllas son calisto themistocles right-hand man aeskylos aegean sea starting battle artemisium ram ships persian ships charge slaughtering several soldiers retreating sinking persian ships following day greeks feign retreat lead group persian ships crevice become stuck greeks charge persian ships cliffs kill persians impressed themistocles skills artemisia brings onto ship sex attempt convince join persians second-in-command refuses causing push aside swear revenge him the persians spill tar sea send suicide bombers swim board greek ships flame bombs artemisia men fire flaming arrows torches ignite tar athenian manages kill one persians falls tar carrying torch causing ships sides explode themistocles thrown sea explosion nearly drowns rescued aeskylos stands scyllas side succumbs injuries believing themistocles dead artemisia forces withdraw recovering injuries themistocles learns hundred warriors six ships survived disastrous attack executed artemisia daxos arcadian general tells themistocles leonidas 300 men killed ephialtes betrays greeks xerxes themistocles returns athens confronts ephialtes deformed spartan traitor reveals xerxes plans attack burn athens ground ephialites regretful actions welcoming death themistocles spares instead warn xerxes greek forces gathering salamis visits gorgo sparta mourning leonidas death ask help overcome grief leaving themistocles returns leonidas sword delivered ephialtes xerxes orders urges gorgo avenge leonidas in athens xerxes army laying waste ephialtes arrives deliver themistocles message upon learning alive artemisia leaves ready entire navy battle xerxes suggests cautious plan still leaves battle ignoring xerxes advice remaining greek ships charge persians ships two armies battle beginning decisive battle salamis themistocles artemisia fight ends stalemate receiving severe injuries at moment gorgo narrating tale spartans arrives battle along ships numerous greek city states including delphi thebes olympia arcadia sparta uniting surrounded persians daxos leads arcadian army themistocles urges artemisia surrender xerxes watching battle cliff turns back acknowledging naval defeat continues march army artemisia tries kill themistocles one last time killed stabs stomach dilios leads greek assault themistocles gorgo take moment silently acknowledge one another alliance remaining persians attempt counter-attack three charge opposing persians whole greek army'>


The following movies are recommended:


<img src='./images/1766.jpg' title='300 2006 american epic war film based 1998 comic series name frank miller lynn varley fictionalized retellings battle thermopylae within persian wars film directed zack snyder miller served executive producer consultant filmed mostly super-imposition chroma key technique help replicate imagery original comic book in 479 bc one year famed battle thermopylae dilios hoplite spartan army begins story depicting life leonidas childhood kingship via spartan doctrine dilios story continues persian messengers arrive gates sparta demanding earth water token submission king xerxes spartans reply killing kicking messengers well leonidas visits ephors proposing strategy drive back numerically superior persians hot gates plan involves building wall order funnel persians narrow pass rocks sea ephors consult oracle decrees sparta go war carneia leonidas angrily departs messenger xerxes appears rewarding ephors covert support although ephors denied permission mobilize sparta army leonidas gathers three hundred best soldiers guise personal bodyguard joined along way arcadians thermopylae construct wall made stones slain persian scouts mortar angering persian emissary stelios elite spartan soldier orders go back persian lines warn xerxes cutting whipping arm meanwhile leonidas encounters ephialtes deformed spartan whose parents fled sparta spare certain infanticide ephialtes asks redeem father name joining leonidas army warning secret path persians could use outflank surround spartans though sympathetic leonidas rejects since deformity physically prevents holding shield high enough potentially compromising phalanx formation ephialtes enraged the battle begins soon spartans refusal lay weapons using hot gates advantage well superior fighting skills spartans repel wave wave advancing persian army lull battle xerxes personally approaches leonidas persuade surrender offering wealth power exchange allegiance leonidas declines mocks xerxes inferior quality fanatical warriors response xerxes sends elite guard immortals later night despite spartans killed heroically defeat immortals slight help arcadians second day xerxes sends new waves armies asia persian city-states including war elephants crush spartans avail meanwhile ephialtes defects xerxes reveals secret path exchange wealth luxury uniform arcadians retreat upon learning ephialtes betrayal spartans stay leonidas orders injured reluctant dilios return sparta tell happened tale victory in sparta queen gorgo tries persuade spartan council send reinforcements aid 300 theron corrupt politician claims owns council threatens queen reluctantly submits sexual demands return help theron disgraces front council gorgo kills rage revealing within robe bag xerxes gold marking betrayal council unanimously agrees send reinforcements third day persians led ephialtes traverse secret path encircling spartans xerxes general demands surrender leonidas seemingly kneels submission allowing stelios leap kill general furious xerxes orders troops attack leonidas rises throws spear xerxes barely missing spear cuts across wounds face proving god-king mortality leonidas remaining spartans fight last man finally succumb arrow barrage dilios back sparta concludes tale council inspired leonidas sacrifice greece mobilizes one year later persians face army 30 000 free greeks led vanguard 10 000 spartans one final speech commemorating 300 dilios head spartan army leads war persians across fields plataea'>,<img src='./images/2171.jpg' title='meet spartans 2008 american parody film directed jason friedberg aaron seltzer similar past movies scary movie date movie epic movie directs parodies various films although references many films tv shows people pop cultural events focuses mainly 2007 film 300 opened 1 us box office despite receiving overwhelmingly negative reviews film stars sean maguire carmen electra kevin sorbo the film opens spartan elder inspecting three babies first ugly talking baby o

## Conclusion

In this project we were able to implement the following:

* Built a web crawler to scrape American movies text and release posters from Wikipedia. The web crawler ran for approximately 7 hours (with a mandatory delay of 3 seconds between each hit) to download the text and images of the movies released after since 2000.

* Using regular expressions, we cleaned the text, and using sklearn packages, we calculated TF-IDF scores.

* Between each pair of movies, we obtained the cosine similarity measure.

* For a given movie _m_, we identified the top 5 relevant movies based on the cosine similarity measure.

* The approach looks promising, since in most of the test cases, the system has found at least 3 relevant movies, out of 5 movies recommended. If this system is combined with other external resources, such as movie ratings (matrix factorization methods like ALS - Alternating Least Squares can be used to predict the ratings a user could give to the movies), then the recommendations quality can improve further. 
